In [ ]:
from datasets import load_dataset
import random
import pandas as pd
import datasets
from IPython.display import display,HTML
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
import numpy as np
import evaluate

dataset= load_dataset("yelp_review_full")
print(f'yelo_review_full dataset loaded.')

tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
print(f'bert-base-cased pretrained  loaded.')

metric=evaluate.load("accuracy")
def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)
    
def compute_metrics(eval_pred):
    logits,labels=eval_pred
    predictions=np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

tokenized_datasets=dataset.map(tokenize_function,batched=True)
print(f'dataset map function updated.')

small_train_dataset=tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset=tokenized_datasets["test"].shuffle(seed=42)
print(f'dataset 1000 items')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=5)
model_dir="./gen/models/bert-base-cased-finetune-yelp"


training_args=TrainingArguments(output_dir=model_dir,
                               evaluation_strategy="epoch",
                               per_device_train_batch_size=16,
                               num_train_epochs=3,
                               logging_steps=30)
print(f'training_args updated')

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)
print(f'trainer updated')

print(f'start to train')
trainer.train()
print(f'Train Completed!')
small_test_dataset=tokenized_datasets["test"].shuffle(seed=64).select(range(100))
print(trainer.evaluate(small_test_dataset))
print(f'Evaluate Completed!')
trainer.save_model(model_dir)
print(f'Save model Completed!')
trainer.save_state()
print(f'Save state Completed!')

/root/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


yelo_review_full dataset loaded.
bert-base-cased pretrained  loaded.
dataset map function updated.
dataset 1000 items


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


training_args updated
trainer updated
start to train


Epoch,Training Loss,Validation Loss
